前節では`mx.ndarray`と`mxnet.autograd`を利用してスクラッチで組んだがもっと簡単に実装する方法あり。  
今回は`gluon`を利用する。

In [1]:
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

# Set the context

今回はコンテクストはCPU

In [2]:
ctx = mx.cpu()

# Build the dataset
前回と同様にデータセットを準備

In [3]:
X = np.random.randn(10000,2)
Y = 2* X[:,0] - 3.4 * X[:,1] + 4.2 + .01 * np.random.normal(size=10000)

# Load the data iterator
バッチ処理のためのイテレータの導入

In [4]:
batch_size = 4
train_data = mx.io.NDArrayIter(X, Y, batch_size, shuffle=True)  # なぜに今回はyも大文字

# Define the model
前回は各パラメータを配置した後、モデルとしてまとめた。  
スクラッチでどんなものか確認する際にはいいことだが、`gluon`を使えば、あらかじめ定義された層からネットワークをまとめることができる

In [5]:
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))

# Initialize parameters
モデルに対して何か操作を施す前に、重みを初期化する必要がある  
MXNetは`mxnet.init`で、共通の初期化関数を提供している

In [6]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# Difine loss
ロスを自身で定義せず、今回は`gluon.loss.L2Loss`を利用する

In [7]:
loss = gluon.loss.L2Loss()

# Optimizer
SGDをスクラッチで書くかわりに、`gluon.Trainer`にパラメータの辞書を渡して利用する

In [8]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

# Execute training loop
これでループを書く全ての手順が整った。  
最初に`epochs`の定義をする  
次にそれぞれの試行で、`train_data` から値とそのラベルのバッチを取得、イテレートしていく

それぞれのバッチの中では以下のような過程を経る  

- `yhat`予測と`loss`損失の生成: 順伝播
- 勾配計算:逆伝播(`loss.backward()`)
- SGDによるモデルのパラメータのアップデート

In [9]:
epochs = 2
ctx = mx.cpu()
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx).reshape((-1,1))
        with autograd.record():
            output = net(data)
            mse = loss(output, label)
        mse.backward()
        trainer.step(data.shape[0])
        
        ##########################
        # ロスの移動平均を保持
        ##########################
        if i == 0:
            moving_loss = np.mean(mse.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(mse.asnumpy()[0])
            
        if i % 500 == 0:
            print('Epoch {}, batch {}. Moving avg of loss: {}'.format(e, i, moving_loss))

Epoch 0, batch 0. Moving avg of loss: 17.40207290649414
Epoch 0, batch 500. Moving avg of loss: 0.11727231408217394
Epoch 0, batch 1000. Moving avg of loss: 0.0008214371748440522
Epoch 0, batch 1500. Moving avg of loss: 5.698019568234263e-05
Epoch 0, batch 2000. Moving avg of loss: 4.182771279906423e-05
Epoch 1, batch 0. Moving avg of loss: 0.00027339678490534425
Epoch 1, batch 500. Moving avg of loss: 5.591434771157916e-05
Epoch 1, batch 1000. Moving avg of loss: 5.126880809587632e-05
Epoch 1, batch 1500. Moving avg of loss: 5.1919817488819594e-05
Epoch 1, batch 2000. Moving avg of loss: 4.179446366995547e-05


> Wとbを取り出す方法がわからない、進めてけばわかる？